# YASA

This notebook demonstrates how to apply the detection only on **NREM sleep** by providing an hypnogram (= sleep stage vector) to YASA.

Please make sure to install YASA first by typing the following line in your terminal or command prompt:

`pip install --upgrade yasa`


## Load the data and hypnogram

We first load a full-night 3-channels dataset (Cz, Fz, Pz) sampled at 100 Hz. The data is in compressed NumPy format (*.npz*).

In [1]:
import yasa
import numpy as np
import pandas as pd
import seaborn as sns
from visbrain.io import read_hypno, oversample_hypno
sns.set(context='notebook', font_scale=1.3)

# Load data
f = np.load('data_full_6hrs_100Hz_Cz+Fz+Pz.npz')
data, ch_names = f['data'], f['chan']
sf = 100.
times = np.arange(data.size) / sf

print(data.shape, ch_names)
print(np.round(data[:, 0:5], 3))

D:\Programs\Anaconda\lib\site-packages\vispy-0.5.3-py3.6.egg\vispy\visuals\isocurve.py:22: UserWarning: VisPy is not yet compatible with matplotlib 2.2+
  warnings.warn("VisPy is not yet compatible with matplotlib 2.2+")


(3, 2161058) ['Cz' 'Fz' 'Pz']
[[15.797 22.307 39.922 25.657 27.094]
 [16.896 26.385 40.966 21.833 24.456]
 [ 5.899 14.297 36.592 26.094 23.395]]


**Hypnogram**

Next, we load the sleep staging vector (a.k.a hypnogram). To do so, we use [Visbrain](http://visbrain.org/sleep)'s built-in functions. Please see [here](http://visbrain.org/sleep.html#hypnogram) to read more about hypnogram format and loading in Visbrain.

In the code below, we first load a 30-seconds encoded stage vector (`hypno_30s`) and over-sample it to have the same size as the data (i.e. one point per data sample). This yields a NumPy array (`hypno`) where:
- 0 = Wake
- 1 = N1 sleep
- 2 = N2 sleep
- 3 = N3 sleep
- 4 = REM sleep

The detection will ONLY work if your hypnogram is formatted EXACTLY like the above, i.e. an integer list or array with the same size as your data and where each integer corresponds to a specific sleep stage.

In [2]:
hypno_30s, _ = read_hypno('data_full_6hrs_100Hz_hypno.hyp')
hypno = oversample_hypno(hypno_30s, data.shape[1])

# Alternatively, if you do not have Visbrain:
# hypno = np.load('data_full_6hrs_100Hz_hypno.npz').get('hypno')

print(hypno.shape)
print('Unique values =', np.unique(hypno))

Hypnogram successfully loaded (data_full_6hrs_100Hz_hypno.hyp)
18-Feb-19 12:33:55 | INFO | Hypnogram successfully loaded (data_full_6hrs_100Hz_hypno.hyp)


(2161058,)
Unique values = [0 1 2 3 4]


*************

**Applying the detection on NREM sleep only**

To apply the multi-channel detection, we use the `yasa.spindles_detect_multi` function. Please check [01_spindles_detection_multi.ipynb](01_spindles_detection_multi.ipynb) for more details. We also pass the hypnogram vector to restrain the detection to NREM sleep epochs (stage 1, 2 and 3).

In [3]:
sp = yasa.spindles_detect_multi(data, sf, ch_names=ch_names, hypno=hypno)
print(sp.shape[0], 'spindles detected.')
sp.head().round(3)

1830 spindles detected.


,Start,End,Duration,Amplitude,RMS,AbsPower,RelPower,Frequency,Oscillations,Symmetry,Stage,Channel,IdxChannel
0,522.56,523.78,1.22,45.810,10.952,2.139,0.475,13.068,16.0,0.179,2,Cz,0
1,585.54,586.38,0.84,59.683,13.335,2.257,0.476,12.958,10.0,0.647,2,Cz,0
2,598.09,599.52,1.43,79.858,16.234,2.377,0.424,12.889,19.0,0.368,2,Cz,0
3,604.37,605.08,0.71,60.637,12.875,2.222,0.355,12.648,9.0,0.500,2,Cz,0
4,607.53,608.05,0.52,50.211,13.847,2.358,0.258,13.452,6.0,0.849,2,Cz,0


In [4]:
# We now print the number of spindles detected per channel and sleep stage
sp.groupby(['Channel', 'Stage'])['Start'].count()

Channel  Stage
Cz       1          1
         2        620
         3         24
Fz       1          1
         2        502
         3         15
Pz       1          2
         2        650
         3         15
Name: Start, dtype: int64

**Let's compare with the detection on the whole recording**

In [5]:
sp = yasa.spindles_detect_multi(data, sf, ch_names=ch_names)
print(sp.shape[0], 'spindles detected.')
sp.head().round(3)

1919 spindles detected.


,Start,End,Duration,Amplitude,RMS,AbsPower,RelPower,Frequency,Oscillations,Symmetry,Channel,IdxChannel
0,522.54,523.79,1.25,45.509,10.850,2.131,0.470,13.083,16.0,0.190,Cz,0
1,585.51,586.38,0.87,59.435,13.185,2.255,0.473,12.946,11.0,0.659,Cz,0
2,598.06,599.54,1.48,79.922,15.977,2.373,0.421,12.889,19.0,0.376,Cz,0
3,604.36,605.12,0.76,60.649,12.520,2.209,0.350,12.646,9.0,0.481,Cz,0
4,607.53,608.05,0.52,50.211,13.847,2.358,0.258,13.452,6.0,0.849,Cz,0
